In [ ]:
import sys
!{sys.executable} -m pip install transformers accelerate datasets[audio] ipywidgets --quiet

In [ ]:
import os
from pathlib import Path
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline

In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32
device

In [ ]:
model_id = "distil-whisper/distil-large-v3"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

In [ ]:
pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=128,
    torch_dtype=torch_dtype,
    device=device,
)

In [ ]:
for filename in os.listdir(os.path.join("data", "inputs")):
  result = pipe(os.path.join("data", "inputs", filename))
  with open(os.path.join("data", "outputs", f"{Path(filename).stem}.txt"), "w") as f:
    f.write(result)
  print(result["text"])